<a href="https://colab.research.google.com/github/daniel-jyc/Impression_Generation_App/blob/main/Capstone_Impression_Generation_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install gradio transformers peft bitsandbytes accelerate sacremoses rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 k

In [2]:
import gradio as gr
import os
import json
import pandas as pd
import torch
import sacremoses
import accelerate
from rouge import Rouge

from pprint import pprint
from getpass import getpass

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM

In [3]:
base_model, new_model = "microsoft/biogpt" , 'rlmjy/ft_new_biogpt'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

finetuned_model = AutoPeftModelForCausalLM.from_pretrained(
    new_model,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map={"":0}
)

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

adapter_config.json:   0%|          | 0.00/484 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/9.49M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

In [4]:
def generate_prompt_test(input):

    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{"Find the impresssion from the background and findings obtained in the radiology examination"}
### Input:
{input}
### Response: """

def get_output(s):
    text = s.split("Response:")[-1].split("</s>")[0].strip()
    return(text)

def make_inference(dataframe, i, context = None):
    rouge_scorer = Rouge()

    dataframe = pd.read_csv(dataframe)
    background = dataframe['clinical information'][i]
    findings = dataframe['findings'][i]
    impression = dataframe['original impression'][i]

    input = background + findings
    prompt = generate_prompt_test(input)

    inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to("cuda:0")
    finetuned_model.generation_config.pad_token_id = finetuned_model.generation_config.eos_token_id

    outputs_1 = finetuned_model.generate(**inputs, max_new_tokens=128,
                                early_stopping=True,
                                num_beams=3,
                                num_return_sequences=3,)

    output_text = get_output(tokenizer.decode(outputs_1[0], skip_special_tokens=True))

    score_single = rouge_scorer.get_scores(
                    hyps= output_text,
                    refs= impression,
                    )
    score = score_single[0]["rouge-l"]["f"]

    return background, findings, output_text, impression, score

In [5]:
demo = gr.Interface(
    fn= make_inference,

    inputs = [gr.File(file_types=[".csv"]), gr.Slider(0, 5, step = 1, label = 'Sample ID')],

    outputs=[gr.Textbox(lines=3, placeholder="Clinical Information", label="Clinical Information"),
        gr.Textbox(lines=3, placeholder="Findings", label="Findings"),
        gr.Textbox(lines=3, placeholder="Generated Impression", label="Generated Impression"),
        gr.Textbox(lines=3, placeholder="Original Impression", label="Original Impression"),
        gr.Number(label="ROUGE-L Score")],
    title="Radiology Impression Generation Demo App",
    description="Submit a csv file including clinical information and radiology findings to automatically generate impression.",
    theme='gradio/soft'
)
demo.launch(share=True)

themes/theme_schema@0.0.3.json:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0eb5af6a2b82cc32ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
